



It's time to pull out some tricks.  

The random forest models are still 
* not performing very well
* and are fairly sensitive to random seed

As I've hypothesized earlier, this seems to suggest that 
* there is an important independent variable (or several) that dictate the performance
* however, this is a very rare event problem, so there are very few events to learn from
* this means that an important independent variable can be improperly split into training,
  validation, and test sets -- that is, without stratifying on these variables, the training
  set does not have a representative distribution of those independent variables
* some models can recover from this (e.g., a linear model that learns a formula)
* unfortunately, this is one of the few weakness of random forests, where a major assumption is
  that the training set is a representative sample of the population at large (especially when
  it comes to extrema -- mins and maxes)

# SMOTE
The imbalanced class ("rare event") problem should have not been overlooked.  However, this
project has changed hands a few times and we have always been on a tight deadline.  At WWE,
I had success in improving machine learning classifiers by using SMOTE ("Synthetic Minority
Over-sampling TEchnique").  This technique is applied to the training set to grow the 
minority class to a similar size as the majority class.  A simpler version of this technique
is to simply oversample the minority: duplicate the minority class records over and over until
you have similarly sized classes.  This can over-emphasize the particular minority examples in
the training set, which SMOTE hedges against a bit by generating synthetic records that look
like minority class records, but not perfectly.

Note that the empahsis on applying this technique to the training set:  there is risk for 
data leakage when using any data augmentation technique.  Namely, if you use SMOTE on the whole data
set before splitting, you are doing it wrong!  This will directly cause leakage ("information pathways")
between the training, validation, and test sets.

Python has a great module dedicated to the imbalanced class problem:
* https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html


You can get your hands on SMOTE quite easily:
```python
from imblearn.over_sampling import SMOTE
```

If you have a lot of categorical variables, it's actually better to use a modified version
of SMOTE for this use case:
```python
from imblearn.over_sampling import SMOTENC
```

For more info, read the original paper on SMOTE:
* N. V. Chawla, K. W. Bowyer, L. O.Hall, W. P. Kegelmeyer, “SMOTE: synthetic minority over-sampling technique,” Journal of artificial intelligence research, 321-357, 2002.


# Parametric Learning
In addition to class imblance, another issue I identified above is tied to 
the dependence of the random forest's performance on random seed.  Though there will
always be some variability when switching seeds, it shouldn't be the difference
between a 0.5 AUC and a 0.6 AUC (this is shown below with random forests).  

A significant dependence on random seed
implies that the model is sensitively dependent on the specifics and details of
the training set.  Specifically,
this implies a dependence on how certain important variables are distributed into
training, validation, and test.  

Some models are robust against this.  For example, assume the target is linearly 
dependent on an important independent variable, and assume that the independent 
variable was poorly split into training and validation, where the training set
sees only the bottom 70% of the variables range, while the training set only
holds instances in the range's top 30%.  A linear regression will learn a linear
formula on the training set, which will still be largely applicable and useful
on the validation set -- even though it never saw instances in the range's top
30%.  However, a random forest will not be able to do this.  Despite all of the
great properties of a random forest, it cannot extrapolate.  

In a rare event
classification problem, this issue is more likely to affect a random forest and
can only be mitigated if you properly stratify the data split on the important 
variables. Note that without this stratification, a signal augmentation technique
like SMOTE will not likely help all that much.


In previous attempts, we tried to mitigate this issue by stratifying on the target variable,
which helped ensure that the training distribution of the target matched the
distribution in validation; but this was not enough!  The poor performance implied
that there might be no signal, but the dependence on random seed implied that
there was -- some important variable(s) exist that, when split correctly ("when
using the right random seed"), improves the model performance.  However, it's not
clear how to identify all the right variables to stratify on.  Instead, given my
hypothesis, it would be better to use a parametric model, which has a chance of 
learning a formula that can extrapolate.  

Why not try logistic regression?

# Missing Indicator Method
One of the great properties of a random forest is that you really do not have to
do much to the data.  For example, it will figure out a linear, quadratic, or
exponential dependence equally well (or any monotonic transformation).  It is
also reasonably well-suited for figuring out conditional relationships (after all,
each tree is basically a heirarchy of if-then statements), which means you do not
necessarily have to worry about adding interactions in explicitly (e.g., x1, x2, and
x1\*x2).  If one assumes that missing data is validly informative (i.e., does not introduce
data leakage), then it can basically be left as-is or coded as some
extreme value way outside the variable's range (e.g.,-999999), whereas this would
destroy a linear model which requires a well-conceived imputation strategy.  

The missing indicator method (MIM) is a technique that is considered terrible
by statisticians interested in unbiased estimates of model coefficients (relationship
strengths), but which is actually top notch when it comes to building a predictive
model where one is less interested in unbiased coefficients and more interested
on predictive performance on held out data sets.  

To implement the MIM, you simply take any column with missing data (call it x) and
make a second column (call it x\_missing).  Wherever x has a missing value, x\_missing
is flagged with a 1, but is 0 otherwise.  Then, the missing values in x are recoded to
0.  

So, say you have a single-variable modes like so:
```
y = m*x+b
```

This now becomes:
```
y = m1*x + m2*x_missing + b
```

As you can see, instead of imputing with the median or mean, this technique will
optimize the best replacement value (i.e., after learning has occurred, you can 
essentially rewrite `m2` as `m2 = m1*x_replacement`; this allows `m1` to 
be learned without forcing an imputed value on it).

Since random forests are fairly robust against variable representation, its performance
shouldn't be affected all that much by choosing to use the MIM (or not).  However, predictive models
that require an imputation strategy will often show incredible performance boosts using
this technique.


# L1 Regularization
The downside to the MIM is that it can double the data dimensionality if every
variable has missing data (which is nearly true for our data set).  This wouldn't be
so bad if every variable was important -- but you can basically assume beforehand that
every variable is not important, and many are probably no better than noise.  This means
that you might be doubling the amount of useless variable, which can cause degradation in
model performance.  

Fortunately, there are techniques for eliminating variables in regressions.  One such technique
in linear regression is called LASSO, which uses the L1 penalty and chooses to zero out variables
that a penalized below a certain threshold.  Importantly, the L1 penalty does most of the
hard work here:  it serves to amplify coefficients of variables with signal and diminish the 
coefficients of noisy, useless variables.  

# Results
In this notebook, I explore:
* SMOTE, SMOTENC
* MIM
* RandomForest -> Logistic Regression
* L1 Penalized Logistic Regression 
    - and for comparison: no penalty, L2 penalty, and ElasticNet penalty


I find that SMOTE can help a random forest, but as assumed, only when the data
split was opportunistic.  For certain "bad" data splits (based on random seed), SMOTE
cannot help the random forest.

I then move on logistic regression, where I automatically include MIM because logistic
regression requires putting some thought into an imputation strategy.  **Starting with
SMOTE and L1 penalty, we find the AUC on validation at 0.995.**  Remember: the highest
AUC we got with "opportunistic" splits using the RF was around 0.58.  

I then investigate the cause of such a dramatic increase in AUC.  I find that SMOTE
is absolutely necessary for this success (without it, AUC suffers significantly).  I also
find that logistic regression performs well in general, independent of solver used: the worst
seen with no penalty was around 0.66 AUC, though most of the solvers got up to 0.83 AUC with
no penalty; L2 penalty has no improvement over 0.83 AUC (and in fact, almost the entire
tradeoff range in ElasticNet performs the same as well); however, nothing comes close
to beating pure L1 penalty.  Finally, by playing with random seeds a bit, I find that
the results are largely independent of random seed.  


# Next Steps
In the next notebook, I will explore variable importances in more detail.  I will
also look into using LIME to understand model decisions at the single-decision level. Finally,
since we found that the sensor data is nearly 100% ineffective (quick glance at importances in
this notebook places them at the tail end), I will use the larger "democlinical patient set"
to build this model on.  This set is about twice the size of the Toitu-restricted set I've been
using.  Importantly, since these ~1500 patients are unseen in this notebook, I should have
a holdout sample from this subset that is tested last...  

Since I do not have a test set in this notebook, these results should be strengthened using
cross validation.

In [2]:
from imblearn.over_sampling import SMOTE, ADASYN, SMOTENC

import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import auc, roc_auc_score, precision_score, average_precision_score, \
    accuracy_score, balanced_accuracy_score, recall_score, confusion_matrix
from scipy.stats import pearsonr, spearmanr
from matplotlib import pyplot as plt
%matplotlib inline

# Create the Data

In [38]:


target_cols = [
    'adscmat_labortype_spontaneousoraugmented_db', 
    'adsc_dlvry_dt', 
    'adsc_dlvrybefore28wks', 
    'adsc_dlvrybefore34wks', 
    'adsc_dlvryga_dys', 
    'adsc_dlvryga_dys_sbadj']

# Get Full Dataset
dff = pd.read_csv('../data/processed/full_set_v4_20200110_KU.csv')

# Set PatId Index
dff.set_index('patid', inplace=True)

# Remove Cesareans/Inductions
dff.query('adscmat_labortype_spontaneousoraugmented_db == 1', inplace=True)

# Remove records w/ GA@Birth < 25 wks (168 days is last day of week 24, so let's do 170 for clean cut off)
dff.query('adsc_dlvryga_dys >= 170', inplace=True)

# NEW: Remove records w/ GA@Recruitment > 25 weeks
dff.query('adelig_ccga <= 170', inplace=True)
dff.drop('adelig_ccga', axis=1, inplace=True)


# Feature Subsets
#  -- Sensor Cols:       Monica and Toitu Cols (usually; we dropped Monica above)
#  -- Toitu Cols:        Toitu Cols
#  -- Toitu F1 Cols:     Toitu F1 Cols
#  -- Toitu F1 Pwr Cols: Toitu F1 Power Cols (band21, band31, band41)
democlinical_cols = [col for col in dff.columns if 'sensor' not in col]
sensor_cols = [col for col in dff.columns if 'sensor' in col]
monica_cols = [col for col in sensor_cols if 'mon' in col]
toitu_cols = [col for col in sensor_cols if 'toi' in col]
toitu_f1_cols = [col for col in toitu_cols if 'f1' in col]
toitu_f1_pwr_cols = [col for col in toitu_f1_cols if 'pwr' in col]
toitu_f1_orthog_pwr_cols = [col for col in toitu_f1_pwr_cols if '41' not in col]
toitu_f3_cols = [col for col in toitu_cols if 'f3' in col]
toitu_f3_pwr_cols = [col for col in toitu_f3_cols if 'pwr' in col]


# NOTE: We are simply dropping all Monica Cols for this analysis
#rmssd_data_quality = [col for col in monica_cols 
#                     if 'rwaves' in col or 'perc' in col or 'epoch' in col]
#dff.drop(rmssd_data_quality, axis=1, inplace=True)
dff.drop(monica_cols, axis=1, inplace=True)


# Patient Subsets
#  -- Toitu F1 (906): patients with all 8 Toitu F1 vars
#  -- Toitu F1 Pwr (1853):  patients with all 3 Toitu F1 Power vars
#  -- Toitu F1 Orthog Power (1853):  patients with power bands 21 and 31
#  -- Toitu F1F3 (389): patients with all F1 and F3 vars
#  -- Toitu F1F3 Pwr (389):  patients with all F1 and F3 Power vars
toitu_f1_population = \
    dff[toitu_f1_cols].replace(-999999,np.nan).isnull().sum(axis=1).map(lambda x: x==0)
toitu_f1_pwr_population = \
    dff[toitu_f1_pwr_cols].replace(-999999,np.nan).isnull().sum(axis=1).map(lambda x: x==0)
toitu_f1_orthog_pwr_population = \
    dff[toitu_f1_orthog_pwr_cols].replace(-999999,np.nan).isnull().sum(axis=1).map(lambda x: x==0)
toitu_f1f3_population = \
    dff[toitu_f1_cols+toitu_f3_cols].replace(-999999,np.nan).isnull().sum(axis=1).map(lambda x: x==0)
toitu_f1f3_pwr_population = \
    dff[toitu_f1_cols+toitu_f3_cols].replace(-999999,np.nan).isnull().sum(axis=1).map(lambda x: x==0)


# Data Subsets (rows_cols_df)
##1
toif1_demo_df = dff.copy().loc[toitu_f1_pwr_population, democlinical_cols]
##2
toif1pwr_toif1pwr_df = dff.copy().loc[toitu_f1_pwr_population, target_cols + toitu_f1_pwr_cols]
##3
toif1pwr_demotoif1pwr_df = dff.copy().\
    loc[toitu_f1_pwr_population, democlinical_cols + toitu_f1_pwr_cols]


### EXTRAS: Only worth checking out if 3 pwr cols do well
toif1orthog_toif1orthog_df = dff.copy().\
    loc[toitu_f1_orthog_pwr_population, target_cols + toitu_f1_orthog_pwr_cols]
toif1orthog_demotoif1orthog_df = dff.copy().\
    loc[toitu_f1_orthog_pwr_population, democlinical_cols + toitu_f1_orthog_pwr_cols]

target_cols = [
    'adscmat_labortype_spontaneousoraugmented_db', 
    'adsc_dlvry_dt', 
    'adsc_dlvrybefore28wks', 
    'adsc_dlvrybefore34wks', 
    'adsc_dlvryga_dys', 
    'adsc_dlvryga_dys_sbadj'
]

# SMOTE
Here, we are still using a RandomForest, but we will be introducing SMOTE.

Set Up:
* Model: RandomForestClassifier
    - 100 trees
* Missing Data
    - Not treated (left as -999999)
* Target: adsc_dlvrybefore34wks
* Data: toif1pwr_demotoif1pwr_df
    - democlinical + Toitu Power Cols
* Col Drop: 
    - remove sensor_toi_f1_mvt_pwr_mvt41 
* SMOTE
    - using regular SMOTE
* Results
    - Terrible: ~0.5 AUC
    - **UPDATE (2020-Jan-27)**: found out that, yet again, these results depend on random seed; for example,
      in my last run, I updated all random seeds to 47 -- and suddenly we have a 0.58 AUC
      here...  except that on scenarios below that had higher AUCs, we now have lower AUCs;
      it's nearly inexplicable...

In [34]:
# INPUTS
target = 'adsc_dlvrybefore34wks'
data = toif1pwr_demotoif1pwr_df
seed=47
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]

# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
sm = SMOTE(random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=seed)
rf.fit(x_trn, y_trn)

# Make Predictions
yp_trn = rf.predict(x_trn)
yp_val = rf.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
imp = zip(x.columns, rf.feature_importances_)
pd.DataFrame(imp, columns=['x_col','gini_importance']).\
    sort_values(by='gini_importance',ascending=False).\
    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

Classication Metrics
Trn Accuracy: 1.0
Val Accuracy: 0.9730215827338129
-------------------
Trn Bal Accuracy: 1.0
Val Bal Accuracy: 0.5873076503328604
-------------------
Trn AUROC: 1.0
Val AUROC: 0.5873076503328604
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.1575327972915785
-------------------
Trn Precision Score: 1.0
Val Precision Score: 0.9677692628505891
-------------------
Trn Recall Score: 1.0
Val Recall Score: 0.9730215827338129
-------------------
Trn Confusion:
 [[1258    0]
 [   0 1258]]
Val Confusion:
 [[538   1]
 [ 14   3]]
-------------------


,x_col,gini_importance
1,alcsmk_t1_avecigs.wk,0.064474
2,alcsmk_t2_avecigs.wk,0.055850
3,admh_hxnt2,0.050961
4,adsc_gender2,0.048630
5,adscmat_educ_combohs4,0.047970
6,admh_hxpt2,0.046879
7,adscmat_empl,0.034887
8,advs_prepregweight,0.029889
9,adafppappa_afpmom,0.028739
10,admh_anemia3,0.027622


# SMOTENC

Set Up:
* Model: RandomForestClassifier
    - 100 trees
* Missing Data
    - Not treated (left as -999999)
* Target: adsc_dlvrybefore34wks
* Data: toif1pwr_demotoif1pwr_df
    - democlinical + Toitu Power Cols
* Col Drop: 
    - remove sensor_toi_f1_mvt_pwr_mvt41 
* SMOTE
    - using SMOTENC
* Result
    - showed modest improvement ~0.5 AUC -> 0.53
    - so it's likely SMOTENC we want, but note that we can do better by properly treating
      missing data (see next section)
    - **UPDATE (2019-Jan-27)**: found out that, yet again, these results depend on random seed; for example,
      in my last run, I updated all random seeds to 47 -- and suddenly we have entirely different
      results (e.g., SMOTENC degrades the AUC of SMOTE for random seed 47)

In [35]:
# INPUTS
target = 'adsc_dlvrybefore34wks'
data = toif1pwr_demotoif1pwr_df
seed=47
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]

# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=seed)
rf.fit(x_trn, y_trn)

# Make Predictions
yp_trn = rf.predict(x_trn)
yp_val = rf.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
imp = zip(x.columns, rf.feature_importances_)
pd.DataFrame(imp, columns=['x_col','gini_importance']).\
    sort_values(by='gini_importance',ascending=False).\
    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


Classication Metrics
Trn Accuracy: 1.0
Val Accuracy: 0.9676258992805755
-------------------
Trn Bal Accuracy: 1.0
Val Bal Accuracy: 0.5560405980574048
-------------------
Trn AUROC: 1.0
Val AUROC: 0.5560405980574048
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.07403724079559881
-------------------
Trn Precision Score: 1.0
Val Precision Score: 0.9552638107300002
-------------------
Trn Recall Score: 1.0
Val Recall Score: 0.9676258992805755
-------------------
Trn Confusion:
 [[1258    0]
 [   0 1258]]
Val Confusion:
 [[536   3]
 [ 15   2]]
-------------------


,x_col,gini_importance
1,adscmat_educ_combohs4,0.100135
2,alcsmk_t2_avecigs.wk,0.058787
3,alcsmk_t1_avecigs.wk,0.048279
4,admh_anemia3,0.047013
5,adscmat_toiletwater2,0.045731
6,advs_prepregbmi,0.033973
7,adsmk_avgnumcighome,0.031529
8,adstai_s_anxiety,0.029696
9,advs_prepregweight,0.028763
10,adsc_gender2,0.028331


# Missing Indicator Method
Note that, though a random forest can handle missing values be coded 
as -999999, most other methods cannot -- and this basically includes SMOTE.  Why?
Because SMOTE is using nearest neighbor methods, and you can bet those -999999 are
throwing things off a bit.  


Set Up:
* Model: RandomForestClassifier
    - 100 trees
* Missing Data
    - Missing Indictor Method (for every colX w/ missing data, another column is
      created, colX_missing, which flags all missing data in colX as 1 and non-missing
      data as 0; the missing data in colX are then zeroed out)
* Target: adsc_dlvrybefore34wks
* Data: toif1pwr_demotoif1pwr_df
    - democlinical + Toitu Power Cols
* Col Drop: 
    - remove sensor_toi_f1_mvt_pwr_mvt41 
* SMOTE
    - using SMOTENC
* Result
    - as would be expected by a RF, the typical 100-trees run gave back
      an AUC of 0.53 (RFs are usually fairly robust against these types of
      representational changes)
    - **UPDATE (2020-Jan-27)**: found out that, yet again, these results depend on random seed; for example,
      in my last run, I updated all random seeds to 47 -- and suddenly we have entirely different
      results (e.g., MIM degrades the AUC instead of maintaining it)

In [29]:
# INPUTS
target = 'adsc_dlvrybefore34wks'
data = toif1pwr_demotoif1pwr_df
seed=47
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=seed)
rf.fit(x_trn, y_trn)

# Make Predictions
yp_trn = rf.predict(x_trn)
yp_val = rf.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
imp = zip(x.columns, rf.feature_importances_)
pd.DataFrame(imp, columns=['x_col','gini_importance']).\
    sort_values(by='gini_importance',ascending=False).\
    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


Classication Metrics
Trn Accuracy: 1.0
Val Accuracy: 0.9694244604316546
-------------------
Trn Bal Accuracy: 1.0
Val Bal Accuracy: 0.5
-------------------
Trn AUROC: 1.0
Val AUROC: 0.5
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.030575539568345324
-------------------
Trn Precision Score: 1.0
Val Precision Score: 0.9397837844832048
-------------------
Trn Recall Score: 1.0
Val Recall Score: 0.9694244604316546
-------------------
Trn Confusion:
 [[1258    0]
 [   0 1258]]
Val Confusion:
 [[539   0]
 [ 17   0]]
-------------------


/Users/kevinurban/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,x_col,gini_importance
1,adscmat_educ_combohs4,0.087860
2,admh_hxft2,0.057540
3,adsc_gender2,0.043230
4,adscmat_toiletwater2,0.029335
5,advs_prepregweight,0.028478
6,advs_prepregbmi,0.026149
7,adstai_s_anxiety,0.025883
8,adstai_t_anxiety,0.023718
9,adscmat_grossincome7,0.022637
10,adsmk_avgnumcighome,0.022536


# SMOTENC, MIM, and some RF Regularization (1)
Everything from the last run, but:
* Ensemble Regularization: trees increased: 100 -> 10k
* Depth Regularization:  max_depth: none -> 3


Set Up:
* Model: RandomForestClassifier
    - 10k trees
* Missing Data
    - Missing Indictor Method (for every colX w/ missing data, another column is
      created, colX_missing, which flags all missing data in colX as 1 and non-missing
      data as 0; the missing data in colX are then zeroed out)
* Target: adsc_dlvrybefore34wks
* Data: toif1pwr_demotoif1pwr_df
    - democlinical + Toitu Power Cols
* Col Drop: 
    - remove sensor_toi_f1_mvt_pwr_mvt41 
* SMOTE
    - using SMOTENC
* Regularization
    - Ensemble Regularization: trees increased: 100 -> 10k
    - Depth Regularization:  max_depth: none -> 3
* RESULTS
    - AUC IMPROVED:  0.53 -> 0.57
    - **UPDATE (2020-Jan-27)**: found out that, yet again, these results depend on random seed; for example,
      in my last run, I updated all random seeds to 47 -- and suddenly we have entirely different
      results 
      * impact here: regularization helps take the AUC back to 0.57 (where it began for seed47 on most naive
        set up)

In [39]:
# INPUTS
target = 'adsc_dlvrybefore34wks'
data = toif1pwr_demotoif1pwr_df
seed=47
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
rf = RandomForestClassifier(n_estimators=10000, random_state=seed,
                            n_jobs=-1, max_depth=3)
rf.fit(x_trn, y_trn)

# Make Predictions
yp_trn = rf.predict(x_trn)
yp_val = rf.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
imp = zip(x.columns, rf.feature_importances_)
pd.DataFrame(imp, columns=['x_col','gini_importance']).\
    sort_values(by='gini_importance',ascending=False).\
    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


Classication Metrics
Trn Accuracy: 0.8656597774244833
Val Accuracy: 0.8453237410071942
-------------------
Trn Bal Accuracy: 0.8656597774244833
Val Bal Accuracy: 0.5784131834552003
-------------------
Trn AUROC: 0.8656597774244833
Val AUROC: 0.5784131834552004
-------------------
Trn AUPRC: 0.8085425177221537
Val AUPRC: 0.04019777476603974
-------------------
Trn Precision Score: 0.8671445357209021
Val Precision Score: 0.9469715844234095
-------------------
Trn Recall Score: 0.8656597774244833
Val Recall Score: 0.8453237410071942
-------------------
Trn Confusion:
 [[1049  209]
 [ 129 1129]]
Val Confusion:
 [[465  74]
 [ 12   5]]
-------------------


,x_col,gini_importance
1,adscmat_educ_combohs4,0.102981
2,adsc_gender2,0.066682
3,admh_hxft2,0.048813
4,adscmat_toiletwater2,0.042232
5,alcsmk_t1_avecigs.wk_missing,0.040882
6,alcsmk_t2_avecigs.wk_missing,0.039613
7,adscmat_grossincome7_missing,0.019226
8,admh_hxmc2,0.018280
9,admh_anemia3,0.016564
10,advs_prepregbmi,0.016064


# SMOTENC, MIM, and some RF Regularization (2)
Tightened Depth Regularization:  3 -> 2


Set Up:
* Model: RandomForestClassifier
    - 10k trees
* Missing Data
    - Missing Indictor Method (for every colX w/ missing data, another column is
      created, colX_missing, which flags all missing data in colX as 1 and non-missing
      data as 0; the missing data in colX are then zeroed out)
* Target: adsc_dlvrybefore34wks
* Data: toif1pwr_demotoif1pwr_df
    - democlinical + Toitu Power Cols
* Col Drop: 
    - remove sensor_toi_f1_mvt_pwr_mvt41 
* SMOTE
    - using SMOTENC
* Regularization
    - Ensemble Regularization: trees increased: 100 -> 10k
    - Depth Regularization:  max_depth: 3 -> 2
* RESULTS
    - **AUC IMPROVED:  0.57 -> 0.60**
    - **UPDATE (2020-Jan-27)**: found out that, yet again, these results depend on random seed; for example,
      in my last run, I updated all random seeds to 47 -- and suddenly we have entirely different
      results 
      * Impact Here:  Instead of AUC improving, it now shot down to 0.5 AUC

In [40]:
# INPUTS
target = 'adsc_dlvrybefore34wks'
data = toif1pwr_demotoif1pwr_df
seed=47
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
rf = RandomForestClassifier(n_estimators=10000, n_jobs=-1, random_state=seed,
                            max_depth=2)
rf.fit(x_trn, y_trn)

# Make Predictions
yp_trn = rf.predict(x_trn)
yp_val = rf.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
imp = zip(x.columns, rf.feature_importances_)
pd.DataFrame(imp, columns=['x_col','gini_importance']).\
    sort_values(by='gini_importance',ascending=False).\
    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


Classication Metrics
Trn Accuracy: 0.794912559618442
Val Accuracy: 0.7122302158273381
-------------------
Trn Bal Accuracy: 0.7949125596184419
Val Bal Accuracy: 0.5097675433809887
-------------------
Trn AUROC: 0.7949125596184421
Val AUROC: 0.5097675433809887
-------------------
Trn AUPRC: 0.7192492979315582
Val AUPRC: 0.031194421625329493
-------------------
Trn Precision Score: 0.8087151278469225
Val Precision Score: 0.9415574240359574
-------------------
Trn Recall Score: 0.794912559618442
Val Recall Score: 0.7122302158273381
-------------------
Trn Confusion:
 [[ 867  391]
 [ 125 1133]]
Val Confusion:
 [[391 148]
 [ 12   5]]
-------------------


,x_col,gini_importance
1,adscmat_educ_combohs4,0.091838
2,adsc_gender2,0.061818
3,alcsmk_t1_avecigs.wk_missing,0.045576
4,alcsmk_t2_avecigs.wk_missing,0.043981
5,adscmat_toiletwater2,0.039206
6,admh_hxft2,0.030137
7,adscmat_grossincome7_missing,0.020792
8,admh_hx_baby_neuraltube2_missing,0.017793
9,admh_hxab2_missing,0.017403
10,admh_hxsidssuid2_missing,0.016997


# SMOTENC, MIM, and some RF Regularization (3)
Tightened Depth Regularization:  2 -> 1
Increased Ensemble Regularziation:
* 1st Run: no increase
* 2nd Run: 10k trees -> 50k trees


Set Up:
* Model: RandomForestClassifier
    - 100 trees
* Missing Data
    - Missing Indictor Method (for every colX w/ missing data, another column is
      created, colX_missing, which flags all missing data in colX as 1 and non-missing
      data as 0; the missing data in colX are then zeroed out)
* Target: adsc_dlvrybefore34wks
* Data: toif1pwr_demotoif1pwr_df
    - democlinical + Toitu Power Cols
* Col Drop: 
    - remove sensor_toi_f1_mvt_pwr_mvt41 
* SMOTE
    - using SMOTENC
* Regularization
    - Ensemble Regularization: trees increased: 
        - 1st Run:  No change (10k trees)
        - 2nd Run:  10k trees -> 50k trees
    - Depth Regularization:  max_depth: 2 -> 1
* RESULTS
    - **TOO MUCH REGULARIZATION:  AUC: 0.6 -> 0.58 in both runs**
    - **UPDATE (2020-Jan-27)**: found out that, yet again, these results depend on random seed; for example,
      in my last run, I updated all random seeds to 47 -- and suddenly we have entirely different
      results 
      * Impact Here:  AUC remaining around 0.5 AUC 

In [41]:
# INPUTS
target = 'adsc_dlvrybefore34wks'
data = toif1pwr_demotoif1pwr_df
seed=47
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
rf = RandomForestClassifier(n_estimators=50000, n_jobs=-1, max_depth=1, random_state=seed)
rf.fit(x_trn, y_trn)

# Make Predictions
yp_trn = rf.predict(x_trn)
yp_val = rf.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
imp = zip(x.columns, rf.feature_importances_)
pd.DataFrame(imp, columns=['x_col','gini_importance']).\
    sort_values(by='gini_importance',ascending=False).\
    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


Classication Metrics
Trn Accuracy: 0.6665341812400636
Val Accuracy: 0.48381294964028776
-------------------
Trn Bal Accuracy: 0.6665341812400636
Val Bal Accuracy: 0.5058932663974681
-------------------
Trn AUROC: 0.6665341812400636
Val AUROC: 0.505893266397468
-------------------
Trn AUPRC: 0.6020954289739409
Val AUPRC: 0.030932606855691918
-------------------
Trn Precision Score: 0.7145239182476096
Val Precision Score: 0.9414419024481908
-------------------
Trn Recall Score: 0.6665341812400636
Val Recall Score: 0.48381294964028776
-------------------
Trn Confusion:
 [[ 541  717]
 [ 122 1136]]
Val Confusion:
 [[260 279]
 [  8   9]]
-------------------


,x_col,gini_importance
1,adscmat_educ_combohs4,0.06954
2,adsc_gender2,0.06272
3,alcsmk_t1_avecigs.wk_missing,0.04974
4,alcsmk_t2_avecigs.wk_missing,0.04882
5,adscmat_toiletwater2,0.03546
6,admh_hx_baby_neuraltube2_missing,0.02466
7,admh_gdmprior2_missing,0.02380
8,admh_hx_baby_iugrsga2_missing,0.02328
9,admh_hxsids2_missing,0.02312
10,admh_hx_baby_cleft2_missing,0.02308


In [43]:
from sklearn.linear_model import Lasso, ElasticNet, LogisticRegression

## MIM w/ LASSO
Here I do a LASSO Regression with rounding.  It's not a recommended technique, but 
something interesting happened:  despite having more false negatives, we have a lot
more true positives.  This motivates a logistic regression w/ L1 Penalty next.


NOTE: the random seed effect is also present here.  For example, when I initially
developed these notebooks, I used random seed 23 in `train_test_split` and 37 in `SMOTENC`. This
resulted in AUC 0.556.  HOWEVER, when I changed both seeds to 47 (like in above RF runs), the
AUC bumped up to 0.597.


In [55]:
# INPUTS
target = 'adsc_dlvrybefore34wks'
data = toif1pwr_demotoif1pwr_df
seed=47
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
lasso = Lasso(alpha=0.1)
lasso.fit(x_trn, y_trn)

# Make Predictions
yp_trn = np.round(lasso.predict(x_trn)).astype(int)
yp_val = np.round(lasso.predict(x_val)).astype(int)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
#imp = zip(x.columns, rf.feature_importances_)
#pd.DataFrame(imp, columns=['x_col','gini_importance']).\
#    sort_values(by='gini_importance',ascending=False).\
#    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


Classication Metrics
Trn Accuracy: 0.7130365659777425
Val Accuracy: 0.6546762589928058
-------------------
Trn Bal Accuracy: 0.7130365659777425
Val Bal Accuracy: 0.5940194259521991
-------------------
Trn AUROC: 0.7135142717766866
Val AUROC: 0.5967477900251009
-------------------
Trn AUPRC: 0.6457150274211438
Val AUPRC: 0.03938653730908509
-------------------
Trn Precision Score: 0.7196169363765884
Val Precision Score: 0.9495004662619262
-------------------
Trn Recall Score: 0.7130365659777425
Val Recall Score: 0.6546762589928058
-------------------
Trn Confusion:
 [[   0    0    0]
 [   1  790  467]
 [   0  254 1004]]
Val Confusion:
 [[  0   0   0]
 [  2 355 182]
 [  0   8   9]]
-------------------


/Users/kevinurban/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
np.unique(yp_trn)

array([-1,  0,  1])

NOTE: Like I said, this technique isn't really a great one.  As you can see, on training, some
targets were rounded down to -1, which we know is impossible.  However, it was still refreshing
to see that there was some signal detected here (AUC 0.56 on seed 23(37) run;  AUC 0.60 on seed47 run).

# MIM, SMOTE, L1-Penalized LogReg
Holy shit. Wtf. This one is amazing.


### UPDATE (2020-Jan-27)
What's crazier is that this is more robust against random seed.  This really indicates to
me that we were experiencing an extrapolation nightmare scenario with the RF.  Basically, if
your training set is a representative sample, then RFs will be amazing; however if any of the
important independent vars or the target var is not representative in training, the RF will
not work on validation -- which is what we saw.


I originally ran this model w/ seed 23 in `train_test_split` and 37 in `SMOTENC`. This
resulted AUC of 0.998 on both training and validation; AUPRC of 0.998 on training, but 0.6
on validation; and a confusion matrix on validation like:
```
[[551   2]
 [  0   3]]
```

It was so good, I was afraid to change the random seed.  But when I change both to 47, we 
get very similar results:  trn/val AUC of 0.9996/0.9955;  tran/val AUPRC of 0.999/0.375;  and
a confusion matrix on validation like:
```
 [[548   5]
 [  0   3]]
```

The results have suffered a little, but only ever-so-slightly.  

When I change the seed to 57, we basically recover that slight decrease:
```
Trn AUROC: 0.9988363072148952
Val AUROC: 0.9981916817359856
-------------------
Trn AUPRC: 0.9976780185758514
Val AUPRC: 0.6
```

Point is, this is the real deal.  The coefficients of the linear model are learning
the important relationships in a way that can extrapolate -- something the RF can't do.  

In [58]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=57
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
model = LogisticRegression(penalty='l1', solver='liblinear')
model.fit(x_trn, y_trn)

# Make Predictions
yp_trn = model.predict(x_trn)
yp_val = model.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
#imp = zip(x.columns, rf.feature_importances_)
#pd.DataFrame(imp, columns=['x_col','gini_importance']).\
#    sort_values(by='gini_importance',ascending=False).\
#    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


Classication Metrics
Trn Accuracy: 0.9988363072148952
Val Accuracy: 0.9964028776978417
-------------------
Trn Bal Accuracy: 0.9988363072148952
Val Bal Accuracy: 0.9981916817359855
-------------------
Trn AUROC: 0.9988363072148952
Val AUROC: 0.9981916817359856
-------------------
Trn AUPRC: 0.9976780185758514
Val AUPRC: 0.6
-------------------
Trn Precision Score: 0.9988390092879257
Val Precision Score: 0.997841726618705
-------------------
Trn Recall Score: 0.9988363072148952
Val Recall Score: 0.9964028776978417
-------------------
Trn Confusion:
 [[1286    3]
 [   0 1289]]
Val Confusion:
 [[551   2]
 [  0   3]]
-------------------


In [59]:
feature_importance = abs(model.coef_[0])
feature_importance = feature_importance / feature_importance.max()
rank = np.argsort(feature_importance)

In [60]:
imps = pd.DataFrame({'ftr': x.columns, 'imp': np.flip(feature_importance[rank]), 'sign': np.sign(np.flip(model.coef_[0][rank]))})

In [61]:
imps

,ftr,imp,sign
0,adafppappa_afpmom,1.000000,1.0
1,adcdrisc_cdrisc_raw,0.864132,1.0
2,addrg_mjlmp2,0.698368,-1.0
3,addrg_mjt12,0.439981,-1.0
4,addrg_mjt22,0.344187,-1.0
5,addrg_meth1yrprior6,0.341854,-1.0
6,addrg_methlmp2,0.293688,1.0
7,addrg_metht12,0.252656,-1.0
8,addrg_metht22,0.160341,-1.0
9,addrg_other1yrprior2,0.149743,-1.0


In [74]:
[(k, imps.iloc[k,:].ftr) for k in imps.index if 'sensor' in imps.iloc[k,:]['ftr']]

[(106, 'sensor_toi_f1_mvt_pwr_mvt21'), (107, 'sensor_toi_f1_mvt_pwr_mvt31')]

# Sanity Check:  What Causes Such Great Performance?
Below we find that:
* SMOTE is absolutely necessary (remove it and the AUC sinks to 0.5)
* L1 regularization is critically necessary
    - use L2 (penalty='l2') and AUC goes from 0.99 down to 0.66
    - remove it (penalty='none') 
    
# 1. Remove SMOTE
0.999 AUC?!?

Let's take out SMOTE and see what happens.

**Spoiler**:  AUC goes from 0.999 -> 0.5.  (SMOTE is amazing, eh?)

These results are robust:
* try un-commenting the SMOTENC lines: the AUC will shoot back up to 0.99
* try changing the random seed:  results will remain similar

In [79]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=47
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
#cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
#sm = SMOTENC(cats, random_state=seed)
#x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
model = LogisticRegression(penalty='l1', solver='liblinear')
model.fit(x_trn, y_trn)

# Make Predictions
yp_trn = model.predict(x_trn)
yp_val = model.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
#imp = zip(x.columns, rf.feature_importances_)
#pd.DataFrame(imp, columns=['x_col','gini_importance']).\
#    sort_values(by='gini_importance',ascending=False).\
#    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

Classication Metrics
Trn Accuracy: 0.9984579799537394
Val Accuracy: 0.987410071942446
-------------------
Trn Bal Accuracy: 0.875
Val Bal Accuracy: 0.49638336347197104
-------------------
Trn AUROC: 0.875
Val AUROC: 0.49638336347197104
-------------------
Trn AUPRC: 0.7515420200462606
Val AUPRC: 0.00539568345323741
-------------------
Trn Precision Score: 0.9984603688306508
Val Precision Score: 0.9891988583046605
-------------------
Trn Recall Score: 0.9984579799537394
Val Recall Score: 0.987410071942446
-------------------
Trn Confusion:
 [[1289    0]
 [   2    6]]
Val Confusion:
 [[549   4]
 [  3   0]]
-------------------


/Users/kevinurban/miniconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [194]:
100* y_val.sum()/y_val.__len__()

adsc_dlvrybefore28wks    0.539568
dtype: float64

In [195]:
len(x.columns)

187

# 2. Swap Out L1 with with other penalties

Here we put SMOTE back in, keep the MIM, but swap out L1 with L2-Penalized LogReg.


**SPOILER**:  
* L2 sucks here.  It's all about the L1 Penalty.
* To be more specific:  AUC on validation goes down to 0.66 (from 0.99)


## 2a: NO PENALTY (no rescaling)
**2020-Jan-27**

The `liblinear` solver does not work for `penalty=none` (logistic regressions default to
L2 penalties in sklearn), so we are forced to explore the other solvers:
* newton-cg
* lbfgs
* sag
* saga

I ran through each of them, recording the results below.  The `newton-cg` method
still did fairly well (AUC on validation of 0.83), though all other solvers did
terrible (worse than chance, i.e., smaller than 0.5 AUC).  From my learnings a week
or so ago, I already knew the SAGA needed variables to be rescaled to promise
convergence; I've not read that this is true for SAG as well.  I'm guessing this is
true for LBFGS as well.  

So, below I report non-scaled results.  Next, I'll rescale all continuous variables
and report again.

Newton-CG
```
Trn AUROC: 1.0
Val AUROC: 0.8306208559373116
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.2684652278177458
```

LBFGS w/o rescaling
```
Trn AUROC: 0.965477114041893
Val AUROC: 0.47106690777576854
-------------------
Trn AUPRC: 0.9374938857783774
Val AUPRC: 0.00539568345323741
```


Sag w/ rescaling
```
Trn AUROC: 0.8855702094647014
Val AUROC: 0.4113924050632911
-------------------
Trn AUPRC: 0.8238578777172731
Val AUPRC: 0.00539568345323741
```

Saga w/o rescaling
```
Trn AUROC: 0.865399534522886
Val AUROC: 0.3987341772151899
-------------------
Trn AUPRC: 0.7994590486456217
Val AUPRC: 0.00539568345323741
```




In [86]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=47
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
# - NOTE: liblinear solver does not support penalty='none'; instead
#   try 'newton-cg', 'lbfgs', 'sag', and/or 'saga' 
model = LogisticRegression(penalty='none', solver='newton-cg')
model.fit(x_trn, y_trn)

# Make Predictions
yp_trn = model.predict(x_trn)
yp_val = model.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
#imp = zip(x.columns, rf.feature_importances_)
#pd.DataFrame(imp, columns=['x_col','gini_importance']).\
#    sort_values(by='gini_importance',ascending=False).\
#    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


Classication Metrics
Trn Accuracy: 1.0
Val Accuracy: 0.9928057553956835
-------------------
Trn Bal Accuracy: 1.0
Val Bal Accuracy: 0.8306208559373116
-------------------
Trn AUROC: 1.0
Val AUROC: 0.8306208559373116
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.2684652278177458
-------------------
Trn Precision Score: 1.0
Val Precision Score: 0.994957500424343
-------------------
Trn Recall Score: 1.0
Val Recall Score: 0.9928057553956835
-------------------
Trn Confusion:
 [[1289    0]
 [   0 1289]]
Val Confusion:
 [[550   3]
 [  1   2]]
-------------------


In [ ]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=47
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
# - NOTE: liblinear solver does not support penalty='none'; instead
#   try 'newton-cg', 'lbfgs', 'sag', and/or 'saga' 
model = LogisticRegression(penalty='none', solver='newton-cg')
model.fit(x_trn, y_trn)

# Make Predictions
yp_trn = model.predict(x_trn)
yp_val = model.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
#imp = zip(x.columns, rf.feature_importances_)
#pd.DataFrame(imp, columns=['x_col','gini_importance']).\
#    sort_values(by='gini_importance',ascending=False).\
#    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

## 2b: NO PENALTY (with rescaled ind vars)
**2020-Jan-27**

Here, we use some rescale methods on the independent variables in order
to get variables on a similar scale...which is a requirement for convergence
for the SAG and SAGA solvers (and maybe LBFGS too).  

LBFGS improves, but SAG and SAGA remain sucky.  This might be due to using
bad stopping conditions... But I'm not sure.  There are related meta parameters
that we can play with to find out:
* `max_iter` is the number of iterations a solver goes through until it is
   assumed that the solver has converged; it defaults to 100
* `tol` is the tolerance for stopping criteria; it defaults to `1e-4`

We can experiment with these things in the next section.  Here, we leave them defaulted.

Newton-CG w/ rescale (default stopping conditions)
```
NCG w/ Standardizer ( (z-avg)/std )
Trn AUROC: 1.0
Val AUROC: 0.8297
-------------------
Val AUROC: 1.0
Val AUPRC: 0.2240

NCG w/ Normalizer ( (z-min)/(max-min) )
Trn AUROC: 1.0
Val AUROC: 0.8288
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.1923
```

LBFGS w/ rescale (default stopping conditions)
* this one is worse on validation, and actually just
  ever so slightly worse on training (it's not perfect)
* these shortcomings might have to do with the solver itself,
  or some meta parameters (stopping conditions)

```
w/ Standardizer
Trn AUROC: 0.9976726144297904
Val AUROC: 0.6621458710066305
-------------------
Trn AUPRC: 0.9965143383682782
Val AUPRC: 0.059152677857713824

w/ Normalizer
Trn AUROC: 0.9584949573312646
Val AUROC: 0.4629294755877034
-------------------
Trn AUPRC: 0.923352435530086
Val AUPRC: 0.00539568345323741
```

SAG w/ rescale (default stopping conditions)
```
w/ Standardizer
Trn AUROC: 0.6586501163692785
Val AUROC: 0.3291139240506329
-------------------
Trn AUPRC: 0.6036276415535662
Val AUPRC: 0.00539568345323741

w/ Normalizer
Trn AUROC: 0.9193173002327386
Val AUROC: 0.4358047016274864
-------------------
Trn AUPRC: 0.8643624049121339
Val AUPRC: 0.00539568345323741
```

SAGA w/ rescale (default stopping conditions)
```
w/ Standardizer
Trn AUROC: 0.6551590380139644
Val AUROC: 0.3381555153707052
-------------------
Trn AUPRC: 0.6020140131824097
Val AUPRC: 0.00539568345323741

w/ Normazlizer
Trn AUROC: 0.9053529868114819
Val AUROC: 0.42495479204339964
-------------------
Trn AUPRC: 0.8482413976043657
Val AUPRC: 0.00539568345323741
```

In [105]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=47
scaler='s'  # standardizer or normalizer
solver='newton-cg'
penalty='none'
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)


# Categoricals & Non-Categoricals
# -- NOTE: this "10" split works on this data set
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
nots = [col for col in x_trn.columns if len(x[col].unique()) >= 10]


# Scale non-cats (nots)
## NOTE (2020-Jan-27): this "0-1" scaler actually hurts 
##   the AUC, while the standardizer does not...(?)
##### NOTE2SELF:  when using SAG or SAGA solvers, you have to scale.
#for col in nots:
#    min_trn = x_trn[col].min()
#    max_trn = x_trn[col].max()
#    scale = lambda z: (z - min_trn) / (max_trn - min_trn)
#    x_trn[col] = x_trn[col].map(scale)
#    x_val[col] = x_val[col].map(scale)
if scaler[0].lower()=='s':
    print('Using Standardizer: (z - avg_trn) / std_trn')
    scale = lambda z: (z - avg_trn) / std_trn
elif scaler[0].lower()=='n':
    print('Using Normalizer: (z - min_trn) / (max_trn - min_trn)')
    scale = lambda z: (z - min_trn) / (max_trn - min_trn)
for col in nots:
    avg_trn = x_trn[col].mean()
    std_trn = x_trn[col].std()
    x_trn[col] = x_trn[col].map(scale)
    x_val[col] = x_val[col].map(scale)


# SMOTE the training data
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
# - NOTE: liblinear solver does not support penalty='none'; instead
#   try 'newton-cg', 'lbfgs', 'sag', and/or 'saga' 
model = LogisticRegression(penalty=penalty, solver=solver)
model.fit(x_trn, y_trn)

# Make Predictions
yp_trn = model.predict(x_trn)
yp_val = model.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
#imp = zip(x.columns, rf.feature_importances_)
#pd.DataFrame(imp, columns=['x_col','gini_importance']).\
#    sort_values(by='gini_importance',ascending=False).\
#    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

Using Standardizer: (z - avg_trn) / std_trn


/Users/kevinurban/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):


Classication Metrics
Trn Accuracy: 1.0
Val Accuracy: 0.9910071942446043
-------------------
Trn Bal Accuracy: 1.0
Val Bal Accuracy: 0.8297166968053044
-------------------
Trn AUROC: 1.0
Val AUROC: 0.8297166968053044
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.22402078337330134
-------------------
Trn Precision Score: 1.0
Val Precision Score: 0.9945945062132112
-------------------
Trn Recall Score: 1.0
Val Recall Score: 0.9910071942446043
-------------------
Trn Confusion:
 [[1289    0]
 [   0 1289]]
Val Confusion:
 [[549   4]
 [  1   2]]
-------------------


## 2b: NO PENALTY (with rescaled ind vars, revised stopping conditions)
**2020-Jan-27**

Here, explore we why SAG and SAGA remain sucky, assuming it might be due to
poorly planned stopping conditions.

Newton-CG works good for everything, so I focus more on LBFGS, SAG, and SAGA.

We have two major hyper parameters that control this:
* `max_iter` is the number of iterations a solver goes through until it is
   assumed that the solver has converged; it defaults to 100
* `tol` is the tolerance for stopping criteria; it defaults to `1e-4`


MAJOR RESULTS
* <font color='red'>though LBFGS remained fairly suboptimal around 0.66 AUC, I was able to
  get the SAG and SAGA solvers to perform at ~0.83 AUC, which is what Newton-CG
  was able to get without rescaling and using default hyperparameters</font>
* <font color='red'>the "normalizer" has dramatically worse performance than the "standardizer"
  on SAG and SAGA </font>
  - it might be salvageable with more hyperparam tweaking, but ... why bother?
  - **ODDLY**, using the Newton-CG solver, the situation is reversed:  the "normalizer"
    works its wonders, while the "standardizer" results in MUCH poorer performance
    

-------------------------------


LBFGS w/ rescale (default stopping conditions)

this one is worse on validation, and actually just ever so slightly worse on training (it's not perfect)
these shortcomings might have to do with the solver itself, or some meta parameters (stopping conditions)

LBFGS
* it is very hard to improve LBFGS
    - below I show one tweak, but I tried MANY, never getting better (if at all)
    
```
w/ Standardizer (from last time, max_iter=100)
Trn AUROC: 0.9976726144297904
Val AUROC: 0.6621458710066305
-------------------
Trn AUPRC: 0.9965143383682782
Val AUPRC: 0.059152677857713824

w/ Standardizer (this time, max_iter=5000)
Trn AUROC: 1.0
Val AUROC: 0.6630500301386376
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.07026378896882494
```


SAG w/ rescale (default stopping conditions)
* was able to get SAG up to ~0.83 AUC, which is what Newton-CG gets
* however, the "normalizer" hurts performance
    - use "standardizer"

```
w/ Standardizer (from last time, max_iter=100, tol=1e-4)
Trn AUROC: 0.6586501163692785
Val AUROC: 0.3291139240506329
-------------------
Trn AUPRC: 0.6036276415535662
Val AUPRC: 0.00539568345323741

*** BETTER ***
w/ Standardizer (max_iter=500, tol=1e-4)
Trn AUROC: 1.0
Val AUROC: 0.6621458710066305
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.059152677857713824


###########################################
#########   THIS ONE   ####################
###########################################
*** BETTER ***
w/ Standardizer (max_iter=5000, tol=1e-4)
Trn AUROC: 1.0
Val AUROC: 0.8288125376732971
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.1922747516272696


*** SAME ***
w/ Standardizer (max_iter=50k, tol=1e-4)
Trn AUROC: 1.0
Val AUROC: 0.8288125376732971
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.1922747516272696

*** WORSE ***  (better than original, but worse than best run)
w/ Standardizer (max_iter=5000, tol=1e-5)
Trn AUROC: 1.0
Val AUROC: 0.6621458710066305
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.059152677857713824

==========================================

w/ Normalizer (max_iter=5000, tol=1e-4)
Trn AUROC: 0.9375484871993794
Val AUROC: 0.6133212778782399      
-------------------
Trn AUPRC: 0.8889655172413793
Val AUPRC: 0.009152677857713828
```




SAGA w/ rescale (default stopping conditions)
* using SAG's best hyperparameters, we also can get SAGA up to ~0.83 AUC, which is what Newton-CG gets
* again, the "normalizer" hurts performance

```
w/ Standardizer (from last time: max_iter=100, tol=1e-4)
Trn AUROC: 0.6551590380139644
Val AUROC: 0.3381555153707052
-------------------
Trn AUPRC: 0.6020140131824097
Val AUPRC: 0.00539568345323741


w/ Standardizer (max_iter=5000, tol=1e-4)
Trn AUROC: 1.0
Val AUROC: 0.8288125376732971  #   <------ NICE
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.1922747516272696

==========================================

w/ Normalizer (max_iter=5000, tol=1e-4)
Trn AUROC: 0.9294026377036462
Val AUROC: 0.6088004822182037  #  <------- Ick! Ew!
-------------------
Trn AUPRC: 0.876274643099932
Val AUPRC: 0.008725327430363403
```


See this post for more info on the Solvers:
* https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions

LBFGS probably is the worst b/c it is an approximation of Newton-CG...but doesn't seem
to have anything else going for it.  The SAG and SAGA methods are stochastic gradient
descent methods, so it's not surprising that (i) we had to normalize, and (ii) they ultimately
work.
    
    
HERE's something weird:  
* though the "standardizer" works so well for SAG and SAGA, while the
  "normalizer" is terrible, the situation is reversed for Newton-CG
  

In [150]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=47
scaler='s'  # standardizer or normalizer
solver='newton-cg'  # liblinear, newton-cg, lbfgs, sag, saga
penalty='none'
tol=1e-4  # default: 1e-4
max_iter=5000 # default: 100
cat_thresh=3
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)


# Categoricals & Non-Categoricals
# -- NOTE: this "10" split works on this data set, but so does "3"; oddly
#      certain numbers will crash the AUC on some solvers...
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < cat_thresh]
nots = [col for col in x_trn.columns if len(x[col].unique()) >= cat_thresh]


# Scale non-cats (nots)
## NOTE (2020-Jan-27): this "0-1" scaler sometimes hurts 
##   the AUC, while the standardizer does not...(?)
##### NOTE2SELF:  when using SAG or SAGA solvers, you have to scale.
if scaler[0].lower()=='s':
    print('Using Standardizer: (z - avg_trn) / std_trn')
    scale = lambda z: (z - avg_trn) / std_trn
elif scaler[0].lower()=='n':
    print('Using Normalizer: (z - min_trn) / (max_trn - min_trn)')
    scale = lambda z: (z - min_trn) / (max_trn - min_trn)
for col in nots:
    avg_trn = x_trn[col].mean()
    std_trn = x_trn[col].std()
    x_trn[col] = x_trn[col].map(scale)
    x_val[col] = x_val[col].map(scale)


# SMOTE the training data
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
# - NOTE: liblinear solver does not support penalty='none'; instead
#   try 'newton-cg', 'lbfgs', 'sag', and/or 'saga' 
model = LogisticRegression(
    penalty=penalty, 
    solver=solver,
    tol = tol,
    max_iter = max_iter,
)
model.fit(x_trn, y_trn)

# Make Predictions
yp_trn = model.predict(x_trn)
yp_val = model.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
#imp = zip(x.columns, rf.feature_importances_)
#pd.DataFrame(imp, columns=['x_col','gini_importance']).\
#    sort_values(by='gini_importance',ascending=False).\
#    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

Using Standardizer: (z - avg_trn) / std_trn


/Users/kevinurban/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):


Classication Metrics
Trn Accuracy: 1.0
Val Accuracy: 0.987410071942446
-------------------
Trn Bal Accuracy: 1.0
Val Bal Accuracy: 0.6621458710066305
-------------------
Trn AUROC: 1.0
Val AUROC: 0.6621458710066305
-------------------
Trn AUPRC: 1.0
Val AUPRC: 0.059152677857713824
-------------------
Trn Precision Score: 1.0
Val Precision Score: 0.9918868541530412
-------------------
Trn Recall Score: 1.0
Val Recall Score: 0.987410071942446
-------------------
Trn Confusion:
 [[1289    0]
 [   0 1289]]
Val Confusion:
 [[548   5]
 [  2   1]]
-------------------


## 2c: L2 PENALTY


For `penalty=none`, the `standardizer` was the go-to scaler for `sag` and
`saga`, while the `normalizer` seemed to be the go-to scaler for `newton-cg`.

For `penalty=l2`, everything seems to prefer the `standardizer`.

So many of the solvers got the SAME EXACT results that I thought something might be wrong, but
then SAG and SAGA gave slightly different (worse) values when using the "normalizer."  I then
also did a sanity check and changed `penalty=l2` to `penalty=l1` for the SAGA solver:
* for "normalizer", I got crappy disheartening results (AUC ~0.60)
* for "standardizer", I recovered the GREAT results that we got using L1 way above (AUC ~0.99)



Newton-CG
```
w/ Standardizer
Trn AUROC: 0.999612102404965
Val AUROC: 0.8288125376732971

w/ Normalizer
Trn AUROC: 0.999612102404965
Val AUROC: 0.6630500301386376
```

Liblinear
```
w/ Standardizer
Trn AUROC: 0.999612102404965
Val AUROC: 0.8288125376732971

w/ Normalizer
Trn AUROC: 0.999612102404965
Val AUROC: 0.6630500301386376
```


LBFGS
```
w/ Standardizer
Trn AUROC: 0.999612102404965
Val AUROC: 0.8288125376732971

w/ Normalizer
Trn AUROC: 0.999612102404965
Val AUROC: 0.6630500301386376
```

SAG
```
w/ Standardizer
Trn AUROC: 0.999612102404965
Val AUROC: 0.8288125376732971

w/ Normalizer
Trn AUROC: 0.9375484871993794
Val AUROC: 0.6133212778782399
```

SAGA
```
w/ Standardizer
Trn AUROC: 0.999612102404965
Val AUROC: 0.8288125376732971

w/ Normalizer
Trn AUROC: 0.9294026377036462
Val AUROC: 0.6088004822182037
```



In [163]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=47
scaler='s'  # standardizer or normalizer
solver='saga'  # liblinear, newton-cg, lbfgs, sag, saga
penalty='l2'
tol=1e-4  # default: 1e-4
max_iter=5000 # default: 100
cat_thresh=3
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)


# Categoricals & Non-Categoricals
# -- NOTE: this "10" split works on this data set, but so does "3"; oddly
#      certain numbers will crash the AUC on some solvers...
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < cat_thresh]
nots = [col for col in x_trn.columns if len(x[col].unique()) >= cat_thresh]


# Scale non-cats (nots)
## NOTE (2020-Jan-27): this "0-1" scaler sometimes hurts 
##   the AUC, while the standardizer does not...(?)
##### NOTE2SELF:  when using SAG or SAGA solvers, you have to scale.
if scaler[0].lower()=='s':
    print('Using Standardizer: (z - avg_trn) / std_trn')
    scale = lambda z: (z - avg_trn) / std_trn
elif scaler[0].lower()=='n':
    print('Using Normalizer: (z - min_trn) / (max_trn - min_trn)')
    scale = lambda z: (z - min_trn) / (max_trn - min_trn)
for col in nots:
    avg_trn = x_trn[col].mean()
    std_trn = x_trn[col].std()
    x_trn[col] = x_trn[col].map(scale)
    x_val[col] = x_val[col].map(scale)


# SMOTE the training data
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
# - NOTE: liblinear solver does not support penalty='none'; instead
#   try 'newton-cg', 'lbfgs', 'sag', and/or 'saga' 
model = LogisticRegression(
    penalty=penalty, 
    solver=solver,
    tol = tol,
    max_iter = max_iter,
)
model.fit(x_trn, y_trn)

# Make Predictions
yp_trn = model.predict(x_trn)
yp_val = model.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
#imp = zip(x.columns, rf.feature_importances_)
#pd.DataFrame(imp, columns=['x_col','gini_importance']).\
#    sort_values(by='gini_importance',ascending=False).\
#    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

Using Standardizer: (z - avg_trn) / std_trn


/Users/kevinurban/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):


Classication Metrics
Trn Accuracy: 0.9996121024049651
Val Accuracy: 0.9892086330935251
-------------------
Trn Bal Accuracy: 0.9996121024049651
Val Bal Accuracy: 0.8288125376732971
-------------------
Trn AUROC: 0.999612102404965
Val AUROC: 0.8288125376732971
-------------------
Trn AUPRC: 0.9992248062015504
Val AUPRC: 0.1922747516272696
-------------------
Trn Precision Score: 0.9996124031007751
Val Precision Score: 0.9943342749687837
-------------------
Trn Recall Score: 0.9996121024049651
Val Recall Score: 0.9892086330935251
-------------------
Trn Confusion:
 [[1288    1]
 [   0 1289]]
Val Confusion:
 [[548   5]
 [  1   2]]
-------------------


## 2D: ElasticNet

Ok, this one is weird... Results aren't changing as I toggle the parameter
between L1 and L2 penalties.

**Update**:  found out for "saga" solver, features must all be of similar scale, so I've 
been trying to normalize things...but still not working....

### UPDATE (2020-Jan-27)
The `penalty='elasticnet'` setting only works with the SAGA solver.

Here, we must introduce the ElasticNet mixing parameter:
* l1_ratio (float, default=None):  the Elastic-Net mixing parameter, with 0 <= l1_ratio <= 1. Only used if   
  `penalty='elasticnet'`. Setting `l1_ratio=0` is equivalent to using `penalty='l2'`, while setting 
  `l1_ratio=1` is equivalent to using `penalty='l1'`. For 0 < l1_ratio <1, the penalty is a combination 
  of L1 and L2.


First thing to check is if it converges on our L1 and L2 results:
```
L1 (l1_ratio=1)
Trn AUROC: 0.9988363072148954
Val AUROC: 0.9954792043399638

L2 (l1_ratio=0)
Trn AUROC: 0.999612102404965
Val AUROC: 0.8288125376732971
```

YES!  Results are reproduced.

Now for more interesting l1 ratios...


Random Seed: 47

| L1 Ratio | Trn AUROC | Val AUROC | Trn AUPRC | Val AUPRC |
|-----------|---------|--------|------------|--------------|
| 1.0 | 0.999 | 0.995 | 1.000 | 0.829 |
| 0.9 | 0.999 | 0.995 | 0.998 | 0.375 |
| 0.8 | 0.999 | 0.829 | 0.998 | 0.192 |
| 0.7 | 0.999 | 0.829 | 0.998 | 0.192 |
| 0.6 | 0.999 | 0.829 | 0.999 | 0.192 |
| 0.5 | 0.999 | 0.829 | 0.999 | 0.192 |
| 0.4 | 1.000 | 0.829 | 0.999 | 0.192 |
| 0.3 | 1.000 | 0.829 | 0.999 | 0.192 |
| 0.2 | 1.000 | 0.829 | 0.999 | 0.192 |
| 0.1 | 1.000 | 0.829 | 0.999 | 0.192 |
| 0.0 | 1.000 | 0.829 | 0.999 | 0.192 |

<font color='red'>Basically, what we find is rapid degradation in performance one we abandon
the L1 penalization.  By the time the L1 Ratio gets to 0.8, the performance
matches that of L2 penalization (`l1_ratio=0`).</font>

In [195]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=47
scaler='s'  # standardizer or normalizer
solver='saga'  # liblinear, newton-cg, lbfgs, sag, saga
penalty='elasticnet'
l1_ratio = 0.1
tol=1e-4  # default: 1e-4
max_iter=5000 # default: 100
cat_thresh=3
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)


# Categoricals & Non-Categoricals
# -- NOTE: this "10" split works on this data set, but so does "3"; oddly
#      certain numbers will crash the AUC on some solvers...
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < cat_thresh]
nots = [col for col in x_trn.columns if len(x[col].unique()) >= cat_thresh]


# Scale non-cats (nots)
## NOTE (2020-Jan-27): this "0-1" scaler sometimes hurts 
##   the AUC, while the standardizer does not...(?)
##### NOTE2SELF:  when using SAG or SAGA solvers, you have to scale.
if scaler[0].lower()=='s':
    print('Using Standardizer: (z - avg_trn) / std_trn')
    avg_trn = x_trn[col].mean()
    std_trn = x_trn[col].std()
    scale = lambda z: (z - avg_trn) / std_trn
elif scaler[0].lower()=='n':
    print('Using Normalizer: (z - min_trn) / (max_trn - min_trn)')
    min_trn = x_trn[col].min()
    max_trn = x_trn[col].max()
    scale = lambda z: (z - min_trn) / (max_trn - min_trn)
for col in nots:
    x_trn[col] = x_trn[col].map(scale)
    x_val[col] = x_val[col].map(scale)


# SMOTE the training data
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
# - NOTE: liblinear solver does not support penalty='none'; instead
#   try 'newton-cg', 'lbfgs', 'sag', and/or 'saga' 
model = LogisticRegression(
    penalty=penalty, 
    solver=solver,
    tol = tol,
    max_iter = max_iter,
    l1_ratio = l1_ratio,
)
model.fit(x_trn, y_trn)

# Make Predictions
yp_trn = model.predict(x_trn)
yp_val = model.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
#imp = zip(x.columns, rf.feature_importances_)
#pd.DataFrame(imp, columns=['x_col','gini_importance']).\
#    sort_values(by='gini_importance',ascending=False).\
#    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

Using Standardizer: (z - avg_trn) / std_trn


/Users/kevinurban/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):


Classication Metrics
Trn Accuracy: 0.9278510473235065
Val Accuracy: 0.8812949640287769
-------------------
Trn Bal Accuracy: 0.9278510473235067
Val Bal Accuracy: 0.6088004822182037
-------------------
Trn AUROC: 0.9278510473235067
Val AUROC: 0.6088004822182037
-------------------
Trn AUPRC: 0.8738983050847458
Val AUPRC: 0.008725327430363403
-------------------
Trn Precision Score: 0.9369491525423729
Val Precision Score: 0.9906359856498321
-------------------
Trn Recall Score: 0.9278510473235065
Val Recall Score: 0.8812949640287769
-------------------
Trn Confusion:
 [[1103  186]
 [   0 1289]]
Val Confusion:
 [[489  64]
 [  2   1]]
-------------------


/Users/kevinurban/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


# Remove the MIM
**2020-Jan-28**

Ok, so we know:
* we must use SMOTENC
* we should absolutely choose pure L1 penalty
* seems fairly robust against random seed (but we should explore this more below)
* MIM seems to be working just fine...but is it necessary?

Here, we will use the best model, but test taking out MIM.

**UPDATE**:  this section was supposed to be about removing the MIM, however I 
quickly found out that there was an odd random seed dependency that remained
(though not as severe as before).

## For Reference:  Pipeline w/ MIM
Here, we just produce the good results with everything in harmony (SMOTENC, MIM,
L1-Penalized Logistic Regression).  

The only thing that changed is where I apply the MIM.  Above, it was being
applied before rescaling, which means though we say "missing values are recoded
as 0," they effectively are recoded as `-mean(colX)/std(colX)`.  From what I read
about MIM, this is actually fine (any constant can be used -- missing column's 
coefficient will just readjust), but conceptually it's a little uglier (there is
something more aesthetic about zeroing out `colX` when `colX_missing` kicks in).  
So, in order to retain the "recode to 0" aesthetic and to have the code match
the Eglish I use to explain what I'm doing, I've moved the application of MIM to
after variables are rescaled.  (This means I have to apply MIM to subsets instead
of the full set at once.)

For some reason, this reduced performance:  `0.995 AUC --> 0.83`.  However, the
performance was able to be picked back up by changing the tolerance:  `1e-4 --> 1e-3`. 
(This is assuming we keep solver as liblinear.)


Actually, after a few model runs and lots of "debugging", I realized that this
model tends to oscillate between 0.999 AUC and 0.83 AUC -- even with the same
parameters set.  This is true at least when using liblinear and using the
default `max_iter` -- in fact, do not even input the default into the model; just leave
the parameter out.  I found when explicitly using `max_iter=100`, the model would
run at 0.83 AUC, but when leaving it out, it would run at either 0.83 AUC or
0.999 AUC....  Very weird....



In [240]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=57
scaler='s'  # standardizer or normalizer or None
solver='saga'  # liblinear, newton-cg, lbfgs, sag, saga
penalty='l1'
l1_ratio = None
tol=1e-4  # default: 1e-4
max_iter=5000 # default: 100
cat_thresh=3
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()


# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]



""""""
# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)
""""""
    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)


# Categoricals & Non-Categoricals
# -- NOTE: this "10" split works on this data set, but so does "3"; oddly
#      certain numbers will crash the AUC on some solvers...
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < cat_thresh]
nots = [col for col in x_trn.columns if len(x[col].unique()) >= cat_thresh]


# Scale non-cats (nots)
## NOTE (2020-Jan-27): this "0-1" scaler sometimes hurts 
##   the AUC, while the standardizer does not...(?)
##### NOTE2SELF:  when using SAG or SAGA solvers, you have to scale.
if scaler is not None:
    if scaler[0].lower()=='s':
        print('Using Standardizer: (z - avg_trn) / std_trn')
        avg_trn = x_trn[col].mean()
        std_trn = x_trn[col].std()
        scale = lambda z: (z - avg_trn) / std_trn
    elif scaler[0].lower()=='n':
        print('Using Normalizer: (z - min_trn) / (max_trn - min_trn)')
        min_trn = x_trn[col].min()
        max_trn = x_trn[col].max()
        scale = lambda z: (z - min_trn) / (max_trn - min_trn)
    for col in nots:
        x_trn[col] = x_trn[col].map(scale)
        x_val[col] = x_val[col].map(scale)


# Employ Missing Indicator Method
#  -- though it's easier to employ MIM before data split, if you want
#     missing data recoded as 0 and missing cols only to have {0,1}, then
#     it's safer to do it after rescaling
'''
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x_trn[col+'_missing'] = [1  if item==-999999 else 0 for item in x_trn[col]]
    x_trn[col] = x_trn[col].replace(-999999, 0) 
    x_val[col+'_missing'] = [1  if item==-999999 else 0 for item in x_val[col]]
    x_val[col] = x_val[col].replace(-999999, 0)
'''

# SMOTE the training data
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
# - NOTE: liblinear solver does not support penalty='none'; instead
#   try 'newton-cg', 'lbfgs', 'sag', and/or 'saga' 
model = LogisticRegression(
    penalty=penalty, 
    solver=solver,
    tol = tol,
    max_iter = max_iter,
    l1_ratio = l1_ratio,
)
#model = LogisticRegression(penalty='l1', solver='liblinear')

model.fit(x_trn, y_trn)

# Make Predictions
yp_trn = model.predict(x_trn)
yp_val = model.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
#imp = zip(x.columns, rf.feature_importances_)
#pd.DataFrame(imp, columns=['x_col','gini_importance']).\
#    sort_values(by='gini_importance',ascending=False).\
#    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

Using Standardizer: (z - avg_trn) / std_trn


/Users/kevinurban/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):


Classication Metrics
Trn Accuracy: 0.9313421256788208
Val Accuracy: 0.8399280575539568
-------------------
Trn Bal Accuracy: 0.9313421256788208
Val Bal Accuracy: 0.7537673297166968
-------------------
Trn AUROC: 0.9313421256788208
Val AUROC: 0.7537673297166967
-------------------
Trn AUPRC: 0.8792633015006821
Val AUPRC: 0.01661337596589395
-------------------
Trn Precision Score: 0.939631650750341
Val Precision Score: 0.9925898765965768
-------------------
Trn Recall Score: 0.9313421256788208
Val Recall Score: 0.8399280575539568
-------------------
Trn Confusion:
 [[1112  177]
 [   0 1289]]
Val Confusion:
 [[465  88]
 [  1   2]]
-------------------


Thought I was going crazy... So I went back to the simpler version of
the most performant model -- the one without need to specify many parameters.

At first, it seemed like this one was incredibly stable @ 0.999 AUC.  But then...every
once in a while...it would hit 0.83 AUC.  I initially thought this had to do with
me trying to add in a parameter, or remove one... But then the performance flip wouldn't
happen every time...  

So I just kept it simple and as-is -- then ran, ran, ran.  Yes, in truth, the AUC flips
between 0.999 and 0.83...

But, why?

Well, there is a random state we didn't specify: the one in the model build.
```
model = LogisticRegression(penalty='l1', solver='liblinear')
```

So, simlarly to the random forest, there is a sensitivity to the opportunistic data split.

One way we can control for this is by specifying the seed...

But we can also keep looking at variable importances between 0.83 AUC and 0.99.


In [297]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=57
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())


# Fit Model
model = LogisticRegression(penalty='l1', solver='liblinear')
model.fit(x_trn, y_trn)

# Make Predictions
yp_trn = model.predict(x_trn)
yp_val = model.predict(x_val)

# Model Metrics
print('Classication Metrics')
print('Trn Accuracy:',accuracy_score(y_trn, yp_trn))
print('Val Accuracy:',accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn Bal Accuracy:',balanced_accuracy_score(y_trn, yp_trn))
print('Val Bal Accuracy:',balanced_accuracy_score(y_val, yp_val))
print('-------------------')
print('Trn AUROC:',roc_auc_score(y_trn, yp_trn))
print('Val AUROC:',roc_auc_score(y_val, yp_val))
print('-------------------')
print('Trn AUPRC:',average_precision_score(y_trn, yp_trn))
print('Val AUPRC:',average_precision_score(y_val, yp_val))
print('-------------------')
print('Trn Precision Score:',precision_score(y_trn, yp_trn, average='weighted'), )
print('Val Precision Score:',precision_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Recall Score:',recall_score(y_trn, yp_trn, average='weighted'), )
print('Val Recall Score:',recall_score(y_val, yp_val, average='weighted'))
print('-------------------')
print('Trn Confusion:\n',confusion_matrix(y_trn, yp_trn), )
print('Val Confusion:\n',confusion_matrix(y_val, yp_val))
print('-------------------')

# Gini Importances
#imp = zip(x.columns, rf.feature_importances_)
#pd.DataFrame(imp, columns=['x_col','gini_importance']).\
#    sort_values(by='gini_importance',ascending=False).\
#    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


Classication Metrics
Trn Accuracy: 0.9992242048099301
Val Accuracy: 0.9964028776978417
-------------------
Trn Bal Accuracy: 0.9992242048099302
Val Bal Accuracy: 0.9981916817359855
-------------------
Trn AUROC: 0.9992242048099302
Val AUROC: 0.9981916817359856
-------------------
Trn AUPRC: 0.9984508133230054
Val AUPRC: 0.6
-------------------
Trn Precision Score: 0.9992254066615027
Val Precision Score: 0.997841726618705
-------------------
Trn Recall Score: 0.9992242048099301
Val Recall Score: 0.9964028776978417
-------------------
Trn Confusion:
 [[1287    2]
 [   0 1289]]
Val Confusion:
 [[551   2]
 [  0   3]]
-------------------


In [398]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=57
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())

while True:
    # Fit Model
    model = LogisticRegression(penalty='l1', solver='liblinear')
    model.fit(x_trn, y_trn)

    # Make Predictions
    yp_trn = model.predict(x_trn)
    yp_val = model.predict(x_val)


    if roc_auc_score(y_val, yp_val) < 0.99:
        feature_importance = abs(model.coef_[0])
        feature_importance = feature_importance / feature_importance.max()
        rank = np.flip(np.argsort(feature_importance))
        imps = pd.DataFrame({
            'ftr': x.columns[rank], 
            'imp': feature_importance[rank], 
            'sign': np.sign(model.coef_[0][rank])
        })
        imps83 = imps[0:30].reset_index().add_suffix('83')
        break

while True:
    # Fit Model
    model = LogisticRegression(penalty='l1', solver='liblinear')
    model.fit(x_trn, y_trn)

    # Make Predictions
    yp_trn = model.predict(x_trn)
    yp_val = model.predict(x_val)


    if roc_auc_score(y_val, yp_val) >= 0.99:
        feature_importance = abs(model.coef_[0])
        feature_importance = feature_importance / feature_importance.max()
        rank = np.flip(np.argsort(feature_importance))
        imps = pd.DataFrame({
            'ftr': x.columns[rank], 
            'imp': feature_importance[rank], 
            'sign': np.sign(model.coef_[0][rank])
        })
        imps99 = imps[0:30].reset_index().add_suffix('99')
        break

# Gini Importances
#imp = zip(x.columns, rf.feature_importances_)
#pd.DataFrame(imp, columns=['x_col','gini_importance']).\
#    sort_values(by='gini_importance',ascending=False).\
#    set_index(pd.Index(range(1,len(x.columns)+1)))[:10]

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


In [400]:
impdf = imps83.merge(imps99, left_on='ftr83', right_on='ftr99')

In [401]:
# Do all var signs agree?  (Yes.)
impdf.apply(lambda x: x.sign83 == x.sign99, axis=1).\
    to_frame('x').\
    query('x == False').\
    __len__()

0

In [402]:
# Since all signs agree, we can technically drop the sign cols.
impdf.drop(['sign83','sign99'], axis=1, inplace=True)

# Also, since we merged on ftr names, we can drop one of the ftr cols
impdf.drop('ftr99', axis=1, inplace=True)

# Re-arrange cols for convenience
impdf = impdf[['ftr83','index83','index99','imp83','imp99']]

In [403]:
impdf.head(20)

,ftr83,index83,index99,imp83,imp99
0,alcsmk_t2_avecigs.wk_missing,0,0,1.000000,1.000000
1,alcsmk_t1_avecigs.wk_missing,1,1,0.880951,0.867162
2,alcsmk_t2_alc_missing,2,2,0.590449,0.488076
3,adscmat_educ_combohs4,3,3,0.513705,0.446070
4,adscmat_grossincome7_missing,4,5,0.468823,0.339978
5,alcsmk_bt2_missing,5,9,0.329940,0.164006
6,adscmat_empl_comb4_missing,6,8,0.318522,0.249989
7,alcsmk_t1_alc_missing,7,7,0.214925,0.268367
8,alcsmk_bt1_missing,8,4,0.206013,0.381125
9,adsc_gender2,9,6,0.178417,0.297203


In [404]:
imp_diff = impdf.apply(lambda x: x.imp83 - x.imp99, axis=1).\
    map(lambda x: abs(x)).\
    to_frame('imp').\
    sort_values(by='imp', ascending=False).\
    reset_index().\
    add_suffix('_diff')
imp_diff.head(10)

,index_diff,imp_diff
0,8,0.175112
1,5,0.165934
2,4,0.128845
3,9,0.118786
4,10,0.112077
5,2,0.102373
6,6,0.068533
7,3,0.067635
8,12,0.061715
9,7,0.053443


In [405]:
impdf.iloc[imp_diff.index_diff].head(10)

,ftr83,index83,index99,imp83,imp99
8,alcsmk_bt1_missing,8,4,0.206013,0.381125
5,alcsmk_bt2_missing,5,9,0.329940,0.164006
4,adscmat_grossincome7_missing,4,5,0.468823,0.339978
9,adsc_gender2,9,6,0.178417,0.297203
10,adscmat_parity,10,16,0.165390,0.053313
2,alcsmk_t2_alc_missing,2,2,0.590449,0.488076
6,adscmat_empl_comb4_missing,6,8,0.318522,0.249989
3,adscmat_educ_combohs4,3,3,0.513705,0.446070
12,adfetalgrowth_deviationindex_missing,12,15,0.121194,0.059479
7,alcsmk_t1_alc_missing,7,7,0.214925,0.268367


Absolute coeff strengths instead of relative.....

In [406]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=57
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())

while True:
    # Fit Model
    model = LogisticRegression(penalty='l1', solver='liblinear')
    model.fit(x_trn, y_trn)

    # Make Predictions
    yp_trn = model.predict(x_trn)
    yp_val = model.predict(x_val)


    if roc_auc_score(y_val, yp_val) < 0.99:
        feature_importance = abs(model.coef_[0])
        rank = np.flip(np.argsort(feature_importance))
        imps = pd.DataFrame({
            'ftr': x.columns[rank], 
            'imp': feature_importance[rank], 
            'sign': np.sign(model.coef_[0][rank])
        })
        imps83 = imps[0:30].reset_index().add_suffix('83')
        break

while True:
    # Fit Model
    model = LogisticRegression(penalty='l1', solver='liblinear')
    model.fit(x_trn, y_trn)

    # Make Predictions
    yp_trn = model.predict(x_trn)
    yp_val = model.predict(x_val)


    if roc_auc_score(y_val, yp_val) >= 0.99:
        feature_importance = abs(model.coef_[0])
        rank = np.flip(np.argsort(feature_importance))
        imps = pd.DataFrame({
            'ftr': x.columns[rank], 
            'imp': feature_importance[rank], 
            'sign': np.sign(model.coef_[0][rank])
        })
        imps99 = imps[0:30].reset_index().add_suffix('99')
        break

        
impdf = imps83.merge(imps99, left_on='ftr83', right_on='ftr99')

# Since all signs agree, we can technically drop the sign cols.
impdf.drop(['sign83','sign99'], axis=1, inplace=True)

# Also, since we merged on ftr names, we can drop one of the ftr cols
impdf.drop('ftr99', axis=1, inplace=True)

# Re-arrange cols for convenience
impdf = impdf[['ftr83','index83','index99','imp83','imp99']]

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


In [414]:
impdf['imp_diff']    = impdf.imp99 - impdf.imp83
impdf['imp_diff_pc'] = np.round(100 * (impdf.imp99 - impdf.imp83) / impdf.imp83, 2)

In [416]:
impdf.head(20)

,ftr83,index83,index99,imp83,imp99,imp_diff,imp_diff_pc
0,alcsmk_t2_avecigs.wk_missing,0,1,4.642583,4.266689,-0.375894,-8.10
1,alcsmk_t1_avecigs.wk_missing,1,0,4.126004,4.659868,0.533864,12.94
2,alcsmk_t2_alc_missing,2,5,2.733655,1.465266,-1.268390,-46.40
3,adscmat_educ_combohs4,3,3,2.245945,2.169745,-0.076200,-3.39
4,adscmat_grossincome7_missing,4,4,1.948538,1.763943,-0.184595,-9.47
5,alcsmk_bt1_missing,5,6,1.590121,1.341765,-0.248356,-15.62
6,adscmat_empl_comb4_missing,6,8,1.350185,1.272548,-0.077637,-5.75
7,adsc_gender2,7,7,1.083397,1.298255,0.214859,19.83
8,alcsmk_t1_alc_missing,8,2,0.941104,2.429169,1.488065,158.12
9,alcsmk_bt2_missing,9,9,0.910539,1.035200,0.124661,13.69


In [417]:
impdf.sort_values(by='imp99', ascending=False).head(20)

,ftr83,index83,index99,imp83,imp99,imp_diff,imp_diff_pc
1,alcsmk_t1_avecigs.wk_missing,1,0,4.126004,4.659868,0.533864,12.94
0,alcsmk_t2_avecigs.wk_missing,0,1,4.642583,4.266689,-0.375894,-8.10
8,alcsmk_t1_alc_missing,8,2,0.941104,2.429169,1.488065,158.12
3,adscmat_educ_combohs4,3,3,2.245945,2.169745,-0.076200,-3.39
4,adscmat_grossincome7_missing,4,4,1.948538,1.763943,-0.184595,-9.47
2,alcsmk_t2_alc_missing,2,5,2.733655,1.465266,-1.268390,-46.40
5,alcsmk_bt1_missing,5,6,1.590121,1.341765,-0.248356,-15.62
7,adsc_gender2,7,7,1.083397,1.298255,0.214859,19.83
6,adscmat_empl_comb4_missing,6,8,1.350185,1.272548,-0.077637,-5.75
9,alcsmk_bt2_missing,9,9,0.910539,1.035200,0.124661,13.69


https://stackoverflow.com/questions/10042388/program-that-convert-html-to-image
    
https://docs.python.org/2/library/subprocess.html

I have been trying to convert a pandas table to an image automatically... which I do below, but
it doesn't really look good.  

In [421]:
import subprocess

In [423]:
impdf.sort_values(by='imp99', ascending=False).head(20).to_html('../reports/figures/test.html')

In [426]:
subprocess.call('wkhtmltoimage -f png --width 0 ../reports/figures/test.html ../reports/figures/test.png', shell=True)

0

Ok, make this all more deterministic...

* Add `random_state` to logistic regression and pass it the seed that everything
else gets passed
* Find a seed that produces AUC 0.99
* Find another seed that produces AUC 0.83

I'll do this in the next JNB:  `07_KU_Random-Seed-Dependence-Demystified`

In [460]:
# INPUTS
target = 'adsc_dlvrybefore28wks'
data = toif1pwr_demotoif1pwr_df
seed=83
# AUC 50: 83
# AUC 66: 5, 17, 53, 61, 67, 73, 79, 89, 127
# AUC 83: 2, 11, 19, 31, 37, 41, 43, 71, 97, 101, 103, 107, 109
# AUC 99: 3, 7, 13, 23, 29, 47, 59, 113, 131
#------------------------------------------------------

# Define Scenario Data (only Demo/Clinical)
x = data.drop(target_cols+['sensor_toi_f1_mvt_pwr_mvt41'], axis=1).copy()
y = data[[target]].copy()

# Only keep records with non-missing target value
valid_target_index = y[target].replace(-999999, np.nan).dropna().index
x = x.loc[valid_target_index,:]
y = y.loc[valid_target_index,:]


# Employ Missing Indicator Method
missing_cols = [
    item for item in 
    x.replace(-999999,np.nan).isna().sum().to_frame('missing').query('missing > 0').index.tolist()
]
for col in missing_cols:
    x[col+'_missing'] = [1  if item==-999999 else 0 for item in x[col]]
    x[col] = x[col].replace(-999999, 0)

    
# Split Data
x_trn, x_val, y_trn, y_val = train_test_split(x, y, train_size=0.7, stratify=y, random_state=seed)

# SMOTE the training data
cats = [idx for idx,col in enumerate(x_trn.columns) if len(x[col].unique()) < 10]
sm = SMOTENC(cats, random_state=seed)
x_trn, y_trn = sm.fit_resample(x_trn.values, y_trn.values.ravel())

# Fit Model
model = LogisticRegression(penalty='l1', solver='liblinear', 
                          random_state=seed)
model.fit(x_trn, y_trn)

# Make Predictions
yp_trn = model.predict(x_trn)
yp_val = model.predict(x_val)


print('AUC:', roc_auc_score(y_val, yp_val))

/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/kevinurban/miniconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


AUC: 0.4972875226039783
